In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
def get_local_alt(f):
    to_local = lambda f : f.replace('/eos/user/e/ekoenig/','/store/user/ekoenig/')
    alt_pattern = to_local(f)

    alt_glob = fc.fs.eos.glob(alt_pattern)
    if any(alt_glob):
        return alt_glob
    
    remote_glob = fc.fs.cernbox.glob(f)
    if any(remote_glob):
        alt_glob = [ to_local(f) for f in remote_glob ]
        remote_glob = [ fc.fs.cernbox.fullpath(f) for f in remote_glob ]
        fc.fs.eos.batch_copy_to(remote_glob, alt_glob)

    alt_glob = fc.fs.eos.glob(alt_pattern)
    return alt_glob

In [30]:

treekwargs = dict(
    use_gen=False,
    treename='Events',
    normalization=None,
)

f_pattern = '/eos/user/e/ekoenig/Ntuples/NanoHH4b/{pairing}_sig_2018_0L/mc/ggHH4b_tree.root'
f_sig = f_pattern.format(pairing='pn_mindiag')

signal = ObjIter([Tree( get_local_alt(f_sig), **treekwargs)])

100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


In [31]:
signal.apply(lambda t : t.reweight(t.genWeight * t.xsecWeight / 1000))

[None]

In [32]:
@cache_variable
def n_loose_btag(t):
    
    nL = t.ak4_h1b1_btag_L + t.ak4_h1b2_btag_L + t.ak4_h2b1_btag_L + t.ak4_h2b2_btag_L
    return ak.values_astype(nL, np.int32)

In [43]:
def set_pn_wp(tree, threshold=0.3):
    tree.extend(
        ak4_h1b1_btag_M = ak.where(tree.ak4_h1b1_bdisc > threshold, 1, 0),
        ak4_h1b2_btag_M = ak.where(tree.ak4_h1b2_bdisc > threshold, 1, 0),
        ak4_h2b1_btag_M = ak.where(tree.ak4_h2b1_bdisc > threshold, 1, 0),
        ak4_h2b2_btag_M = ak.where(tree.ak4_h2b2_bdisc > threshold, 1, 0),
    )

def n_medium_btag(t):
    nM = t.ak4_h1b1_btag_M + t.ak4_h1b2_btag_M + t.ak4_h2b1_btag_M + t.ak4_h2b2_btag_M
    return ak.values_astype(nM, np.int32)

In [52]:
signal.apply(lambda t : set_pn_wp(t, threshold=0.3))

study.quick(
    signal,
    varlist=[n_medium_btag],
)

[None]